# Seeing Different: Identifying Causal Relationships Others Miss

> *"While others see complexity, innovators see clarity"* - This tutorial develops your causal intuition to spot hidden relationships, unmask confounders, and question conventional wisdom.

## The Art of Causal Vision

Most analysts stop at correlation. They see patterns in data and assume they understand causality. **Innovators see deeper.** They question assumptions, hunt for hidden confounders, and discover causal mechanisms others miss entirely.

### What You'll Develop

- **Causal intuition** to spot spurious relationships
- **Confounder detection** skills to unmask hidden variables
- **Mechanism thinking** to understand how causality flows
- **Diagnostic tools** to test your causal theories
- **Skeptical mindset** to question apparent relationships

### The Challenge

Consider these "obvious" relationships:
- Ice cream sales are correlated with drowning deaths
- Countries with more storks have higher birth rates
- Students who take notes on laptops perform worse
- Companies with diverse boards are more profitable

**What's really going on here? Let's find out.**

In [ ]:
# Setup: The Detective's Toolkit
import os
import sys
import warnings

warnings.filterwarnings("ignore")

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
if project_root not in sys.path:
    sys.path.insert(0, os.path.join(project_root, "libs"))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Our causal detection arsenal
from causal_inference.core.base import CovariateData, OutcomeData, TreatmentData
from causal_inference.diagnostics.balance import check_covariate_balance
from causal_inference.estimators.aipw import AIPWEstimator
from causal_inference.estimators.causal_forest import CausalForestEstimator
from causal_inference.estimators.g_computation import GComputationEstimator

# Set up our detective workspace
np.random.seed(42)
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("Set1")

print("🔍 Detective Mode: ACTIVATED")
print("🧠 Causal Vision: ENHANCED")
print("💡 Ready to see what others miss!")

## Case Study 1: The Ice Cream Paradox

**The Apparent Relationship**: Ice cream sales are strongly correlated with drowning deaths. Does ice cream consumption cause drowning?

**The Obvious Answer**: No, that's ridiculous!

**The Innovation**: Let's prove it systematically and learn how to detect such spurious relationships in less obvious cases.

In [ ]:
# Case Study 1: The Ice Cream Paradox - Unmasking Spurious Correlation


def generate_ice_cream_drowning_data(n_months=60):
    """
    Generate data showing spurious correlation between ice cream and drowning
    True confounder: Temperature/Season
    """
    months = np.arange(n_months)

    # Seasonal pattern: temperature varies sinusoidally
    temperature = (
        60 + 25 * np.sin(2 * np.pi * months / 12) + np.random.normal(0, 3, n_months)
    )

    # Ice cream sales driven by temperature (and marketing)
    marketing_spend = np.random.normal(1000, 200, n_months)  # Random marketing
    ice_cream_sales = (
        50  # Base sales
        + 3 * temperature  # Temperature drives sales
        + 0.02 * marketing_spend  # Marketing effect
        + np.random.normal(0, 10, n_months)
    )  # Random variation

    # Drowning deaths driven by temperature/season (NOT ice cream!)
    # More people swim when it's warm -> more drowning risk
    swimming_activity = 10 + 0.5 * temperature + np.random.normal(0, 2, n_months)
    drowning_deaths = (
        2  # Base rate
        + 0.08 * swimming_activity  # Swimming activity causes drowning
        + 0 * ice_cream_sales  # ICE CREAM DOES NOT CAUSE DROWNING!
        + np.random.poisson(1, n_months)
    )  # Random variation

    # Create seasonal indicators
    season = np.array(
        [
            "Winter",
            "Winter",
            "Spring",
            "Spring",
            "Spring",
            "Summer",
            "Summer",
            "Summer",
            "Fall",
            "Fall",
            "Fall",
            "Winter",
        ]
    )
    season_labels = [season[month % 12] for month in months]

    return pd.DataFrame(
        {
            "month": months,
            "temperature": temperature,
            "ice_cream_sales": ice_cream_sales,
            "drowning_deaths": drowning_deaths,
            "marketing_spend": marketing_spend,
            "swimming_activity": swimming_activity,
            "season": season_labels,
        }
    )


# Generate the data
ice_cream_data = generate_ice_cream_drowning_data(60)

print("🍦 Ice Cream and Drowning Dataset Generated")
print("Data spans 5 years (60 months) with seasonal variation")
print("\nData preview:")
print(ice_cream_data.head())

# Calculate the spurious correlation
spurious_correlation = np.corrcoef(
    ice_cream_data["ice_cream_sales"], ice_cream_data["drowning_deaths"]
)[0, 1]

print("\n🚨 SPURIOUS CORRELATION DETECTED!")
print(f"📊 Ice cream sales vs drowning deaths: r = {spurious_correlation:.3f}")
print("📈 This appears to be a strong positive relationship!")
print("🤔 But we know ice cream doesn't cause drowning...")

# Visualize the apparent relationship
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Spurious correlation plot
axes[0, 0].scatter(
    ice_cream_data["ice_cream_sales"],
    ice_cream_data["drowning_deaths"],
    alpha=0.7,
    s=50,
)
z = np.polyfit(ice_cream_data["ice_cream_sales"], ice_cream_data["drowning_deaths"], 1)
p = np.poly1d(z)
axes[0, 0].plot(
    ice_cream_data["ice_cream_sales"],
    p(ice_cream_data["ice_cream_sales"]),
    "r--",
    alpha=0.8,
    linewidth=2,
)
axes[0, 0].set_title(
    f"SPURIOUS: Ice Cream vs Drowning\nr = {spurious_correlation:.3f} (MISLEADING!)"
)
axes[0, 0].set_xlabel("Ice Cream Sales")
axes[0, 0].set_ylabel("Drowning Deaths")

# Time series revealing the pattern
ax1 = axes[0, 1]
ax2 = ax1.twinx()

line1 = ax1.plot(
    ice_cream_data["month"],
    ice_cream_data["ice_cream_sales"],
    "b-",
    label="Ice Cream Sales",
    linewidth=2,
)
line2 = ax2.plot(
    ice_cream_data["month"],
    ice_cream_data["drowning_deaths"],
    "r-",
    label="Drowning Deaths",
    linewidth=2,
)

ax1.set_xlabel("Month")
ax1.set_ylabel("Ice Cream Sales", color="b")
ax2.set_ylabel("Drowning Deaths", color="r")
ax1.set_title("Time Series: Both Follow Seasonal Pattern")

# The true confounder: Temperature
temp_ice_corr = np.corrcoef(
    ice_cream_data["temperature"], ice_cream_data["ice_cream_sales"]
)[0, 1]
temp_drown_corr = np.corrcoef(
    ice_cream_data["temperature"], ice_cream_data["drowning_deaths"]
)[0, 1]

axes[1, 0].scatter(
    ice_cream_data["temperature"],
    ice_cream_data["ice_cream_sales"],
    alpha=0.7,
    color="blue",
    label=f"Ice Cream (r={temp_ice_corr:.2f})",
)
axes[1, 0].scatter(
    ice_cream_data["temperature"],
    ice_cream_data["drowning_deaths"] * 20,
    alpha=0.7,
    color="red",
    label=f"Drowning×20 (r={temp_drown_corr:.2f})",
)
axes[1, 0].set_title("TRUE CAUSE: Temperature Drives Both")
axes[1, 0].set_xlabel("Temperature (°F)")
axes[1, 0].set_ylabel("Value")
axes[1, 0].legend()

# Seasonal breakdown
seasonal_means = ice_cream_data.groupby("season")[
    ["ice_cream_sales", "drowning_deaths"]
].mean()
x = np.arange(len(seasonal_means))
width = 0.35

bars1 = axes[1, 1].bar(
    x - width / 2,
    seasonal_means["ice_cream_sales"] / 10,
    width,
    label="Ice Cream Sales (÷10)",
    alpha=0.8,
)
bars2 = axes[1, 1].bar(
    x + width / 2,
    seasonal_means["drowning_deaths"],
    width,
    label="Drowning Deaths",
    alpha=0.8,
)

axes[1, 1].set_title("Seasonal Pattern: Both Peak in Summer")
axes[1, 1].set_xlabel("Season")
axes[1, 1].set_ylabel("Value")
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(seasonal_means.index)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\n🔍 DETECTIVE INSIGHTS:")
print(f"🌡️ Temperature ↔ Ice Cream: r = {temp_ice_corr:.3f}")
print(f"🌡️ Temperature ↔ Drowning: r = {temp_drown_corr:.3f}")
print(f"🍦 Ice Cream ↔ Drowning: r = {spurious_correlation:.3f} (SPURIOUS!)")
print("\n💡 The real story: Temperature causes both ice cream sales AND drowning!")

In [ ]:
# Unmasking the Spurious Relationship: Controlling for the Confounder

print("🕵️ CAUSAL INVESTIGATION: Controlling for Temperature")

# Prepare data for causal analysis
treatment = TreatmentData(
    values=ice_cream_data["ice_cream_sales"],
    name="ice_cream_sales",
    treatment_type="continuous",
)

outcome = OutcomeData(
    values=ice_cream_data["drowning_deaths"],
    name="drowning_deaths",
    outcome_type="continuous",
)

# The key: Include temperature as a confounder
covariates = CovariateData(
    values=ice_cream_data[["temperature", "marketing_spend"]],
    names=["temperature", "marketing_spend"],
)

# Estimate without controlling for confounders (BIASED)
naive_estimator = GComputationEstimator(model_type="linear", bootstrap_samples=100)
naive_estimator.fit(treatment, outcome)  # No covariates!
naive_effect = naive_estimator.estimate_ate()

print("\n❌ NAIVE ESTIMATE (ignoring confounders):")
print(
    f"📊 Effect of 1 unit ice cream sales: {naive_effect.ate:.4f} more drowning deaths"
)
print(
    f"📈 95% CI: [{naive_effect.confidence_interval[0]:.4f}, {naive_effect.confidence_interval[1]:.4f}]"
)
print("🚨 This suggests ice cream causes drowning! (WRONG!)")

# Estimate controlling for confounders (UNBIASED)
controlled_estimator = GComputationEstimator(model_type="linear", bootstrap_samples=100)
controlled_estimator.fit(treatment, outcome, covariates)  # Include covariates!
controlled_effect = controlled_estimator.estimate_ate()

print("\n✅ CONTROLLED ESTIMATE (accounting for temperature):")
print(
    f"📊 Effect of 1 unit ice cream sales: {controlled_effect.ate:.4f} more drowning deaths"
)
print(
    f"📈 95% CI: [{controlled_effect.confidence_interval[0]:.4f}, {controlled_effect.confidence_interval[1]:.4f}]"
)

if abs(controlled_effect.ate) < 0.01:  # Near zero
    print("✨ REVELATION: The effect disappears when we control for temperature!")
    print("🎯 Ice cream does NOT cause drowning!")
else:
    print("🤔 Small residual effect remains - might be other confounders")

# Demonstrate with partial correlation

# Partial correlation: Ice cream and drowning, controlling for temperature
# This is equivalent to correlating the residuals after regressing out temperature

# Regress ice cream on temperature
ice_cream_temp_coef = np.polyfit(
    ice_cream_data["temperature"], ice_cream_data["ice_cream_sales"], 1
)
ice_cream_residuals = ice_cream_data["ice_cream_sales"] - np.polyval(
    ice_cream_temp_coef, ice_cream_data["temperature"]
)

# Regress drowning on temperature
drowning_temp_coef = np.polyfit(
    ice_cream_data["temperature"], ice_cream_data["drowning_deaths"], 1
)
drowning_residuals = ice_cream_data["drowning_deaths"] - np.polyval(
    drowning_temp_coef, ice_cream_data["temperature"]
)

# Partial correlation
partial_correlation = np.corrcoef(ice_cream_residuals, drowning_residuals)[0, 1]

print("\n🧮 PARTIAL CORRELATION ANALYSIS:")
print(f"📊 Raw correlation: {spurious_correlation:.4f} (MISLEADING)")
print(
    f"🎯 Partial correlation (controlling for temperature): {partial_correlation:.4f}"
)

reduction = abs(spurious_correlation) - abs(partial_correlation)
print(
    f"⬇️ Spurious correlation reduced by: {reduction:.4f} ({reduction/abs(spurious_correlation)*100:.1f}%)"
)

# Visualize the revelation
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Before: Spurious correlation
axes[0].scatter(
    ice_cream_data["ice_cream_sales"],
    ice_cream_data["drowning_deaths"],
    alpha=0.7,
    s=50,
)
z = np.polyfit(ice_cream_data["ice_cream_sales"], ice_cream_data["drowning_deaths"], 1)
p = np.poly1d(z)
axes[0].plot(
    ice_cream_data["ice_cream_sales"],
    p(ice_cream_data["ice_cream_sales"]),
    "r--",
    alpha=0.8,
    linewidth=2,
)
axes[0].set_title(
    f"Before: Raw Correlation\nr = {spurious_correlation:.3f} (MISLEADING!)",
    color="red",
    fontweight="bold",
)
axes[0].set_xlabel("Ice Cream Sales")
axes[0].set_ylabel("Drowning Deaths")

# After: Controlling for temperature
axes[1].scatter(ice_cream_residuals, drowning_residuals, alpha=0.7, s=50)
if abs(partial_correlation) > 0.1:
    z = np.polyfit(ice_cream_residuals, drowning_residuals, 1)
    p = np.poly1d(z)
    axes[1].plot(
        ice_cream_residuals, p(ice_cream_residuals), "g--", alpha=0.8, linewidth=2
    )
axes[1].axhline(y=0, color="black", linestyle="-", alpha=0.3)
axes[1].axvline(x=0, color="black", linestyle="-", alpha=0.3)
axes[1].set_title(
    f"After: Controlling for Temperature\nr = {partial_correlation:.3f} (TRUE RELATIONSHIP!)",
    color="green",
    fontweight="bold",
)
axes[1].set_xlabel("Ice Cream Sales (residuals)")
axes[1].set_ylabel("Drowning Deaths (residuals)")

plt.tight_layout()
plt.show()

print("\n🎉 CAUSAL DETECTIVE SUCCESS!")
print("🔍 We unmasked the spurious relationship!")
print("🌡️ Temperature was the hidden confounder all along!")
print("💡 This is how we 'see different' - by looking for hidden variables!")

## Case Study 2: The Laptop Learning Paradox

**The Observed Pattern**: Students who take notes on laptops perform worse than those using pen and paper.

**The Quick Conclusion**: Laptops hurt learning!

**The Innovation**: Let's dig deeper and discover the hidden mechanisms at work.

In [ ]:
# Case Study 2: The Laptop Learning Paradox - Hidden Mechanisms


def generate_laptop_learning_data(n_students=500):
    """
    Generate student performance data with hidden confounders
    """
    # Student characteristics (hidden confounders)
    baseline_ability = np.random.normal(75, 15, n_students)  # Base academic ability
    tech_comfort = np.random.beta(2, 3, n_students)  # Tech comfort (0-1)
    engagement_level = np.random.normal(0.7, 0.2, n_students)  # Class engagement

    # Course characteristics
    course_difficulty = np.random.choice(
        ["Easy", "Medium", "Hard"], n_students, p=[0.3, 0.4, 0.3]
    )
    course_type = np.random.choice(
        ["Math", "Literature", "Science"], n_students, p=[0.33, 0.33, 0.34]
    )

    # Note-taking method choice (NOT random!)
    # Students choose laptops based on tech comfort and course type
    laptop_propensity = (
        0.3  # Base propensity
        + 0.4 * tech_comfort  # Tech-comfortable students prefer laptops
        + 0.2 * (course_type == "Math")  # Math students prefer laptops
        + -0.1 * (course_difficulty == "Hard")  # Avoid laptops in hard courses
        + np.random.normal(0, 0.15, n_students)
    )  # Random variation

    uses_laptop = (laptop_propensity > 0.5).astype(int)

    # True causal mechanisms:
    # 1. Laptops reduce deep processing (direct effect: -3 points)
    # 2. BUT laptops help with organization (+2 points for organized students)
    # 3. Distraction effect depends on engagement (-5 points for low engagement)

    laptop_direct_effect = -3  # Reduced deep processing
    organization_benefit = 2 * tech_comfort  # Organized students benefit
    distraction_penalty = -5 * (
        engagement_level < 0.5
    )  # Low engagement = more distraction

    # Final test scores
    test_score = (
        baseline_ability  # Base ability
        + laptop_direct_effect * uses_laptop  # Direct laptop effect
        + organization_benefit * uses_laptop  # Organization benefit
        + distraction_penalty * uses_laptop  # Distraction penalty
        + 10 * engagement_level  # Engagement helps performance
        + -5 * (course_difficulty == "Hard")  # Hard courses lower scores
        + 5 * (course_type == "Math") * tech_comfort  # Tech helps in math
        + np.random.normal(0, 5, n_students)
    )  # Random noise

    test_score = np.clip(test_score, 0, 100)  # Realistic score range

    return pd.DataFrame(
        {
            "uses_laptop": uses_laptop,
            "test_score": test_score,
            "baseline_ability": baseline_ability,
            "tech_comfort": tech_comfort,
            "engagement_level": engagement_level,
            "course_difficulty": course_difficulty,
            "course_type": course_type,
            "laptop_propensity": laptop_propensity,
        }
    )


# Generate the data
laptop_data = generate_laptop_learning_data(500)

print("💻 Laptop Learning Dataset Generated")
print("Students choose laptops based on tech comfort and course characteristics")
print("\nData preview:")
print(laptop_data.head())

# Calculate the observed correlation
laptop_users = laptop_data[laptop_data["uses_laptop"] == 1]["test_score"]
paper_users = laptop_data[laptop_data["uses_laptop"] == 0]["test_score"]
observed_difference = laptop_users.mean() - paper_users.mean()

print("\n📊 OBSERVED PATTERN:")
print(f"📝 Paper users average score: {paper_users.mean():.1f}")
print(f"💻 Laptop users average score: {laptop_users.mean():.1f}")
print(f"📉 Raw difference: {observed_difference:.1f} points")

if observed_difference < 0:
    print("🚨 Laptops appear to HURT performance!")
else:
    print("✨ Laptops appear to HELP performance!")

print("🤔 But is this the whole story?")

# Visualize the apparent relationship and hidden patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw comparison
axes[0, 0].boxplot([paper_users, laptop_users], labels=["Paper", "Laptop"])
axes[0, 0].set_title(
    f"Raw Comparison\nLaptops appear worse by {abs(observed_difference):.1f} points"
)
axes[0, 0].set_ylabel("Test Score")

# Selection bias: Who chooses laptops?
laptop_choosers = laptop_data[laptop_data["uses_laptop"] == 1]
paper_choosers = laptop_data[laptop_data["uses_laptop"] == 0]

axes[0, 1].scatter(
    laptop_choosers["tech_comfort"],
    laptop_choosers["baseline_ability"],
    alpha=0.6,
    label="Laptop Users",
    s=30,
)
axes[0, 1].scatter(
    paper_choosers["tech_comfort"],
    paper_choosers["baseline_ability"],
    alpha=0.6,
    label="Paper Users",
    s=30,
)
axes[0, 1].set_title("Selection Bias: Who Chooses Laptops?")
axes[0, 1].set_xlabel("Tech Comfort")
axes[0, 1].set_ylabel("Baseline Ability")
axes[0, 1].legend()

# Engagement levels by note-taking method
axes[1, 0].boxplot(
    [paper_choosers["engagement_level"], laptop_choosers["engagement_level"]],
    labels=["Paper", "Laptop"],
)
axes[1, 0].set_title("Engagement Levels by Method")
axes[1, 0].set_ylabel("Engagement Level")

# Course type distribution
course_crosstab = pd.crosstab(
    laptop_data["course_type"], laptop_data["uses_laptop"], normalize="columns"
)
course_crosstab.plot(kind="bar", ax=axes[1, 1], stacked=True)
axes[1, 1].set_title("Course Type Distribution")
axes[1, 1].set_xlabel("Course Type")
axes[1, 1].set_ylabel("Proportion")
axes[1, 1].legend(["Paper", "Laptop"])
axes[1, 1].tick_params(axis="x", rotation=0)

plt.tight_layout()
plt.show()

# Analyze selection patterns
print("\n🔍 SELECTION BIAS ANALYSIS:")
print(f"💻 Laptop users - Tech comfort: {laptop_choosers['tech_comfort'].mean():.2f}")
print(f"📝 Paper users - Tech comfort: {paper_choosers['tech_comfort'].mean():.2f}")
print(
    f"💻 Laptop users - Baseline ability: {laptop_choosers['baseline_ability'].mean():.1f}"
)
print(
    f"📝 Paper users - Baseline ability: {paper_choosers['baseline_ability'].mean():.1f}"
)
print(f"💻 Laptop users - Engagement: {laptop_choosers['engagement_level'].mean():.2f}")
print(f"📝 Paper users - Engagement: {paper_choosers['engagement_level'].mean():.2f}")
print("\n🎯 Key insight: Laptop and paper users are systematically different!")

In [ ]:
# Causal Analysis: Accounting for Selection Bias

print("🕵️ CAUSAL INVESTIGATION: Controlling for Student Characteristics")

# Prepare data for causal analysis
treatment = TreatmentData(
    values=laptop_data["uses_laptop"], name="uses_laptop", treatment_type="binary"
)

outcome = OutcomeData(
    values=laptop_data["test_score"], name="test_score", outcome_type="continuous"
)

# Create comprehensive covariate set
covariates_df = laptop_data[
    ["baseline_ability", "tech_comfort", "engagement_level"]
].copy()

# Add dummy variables for categorical variables
course_dummies = pd.get_dummies(laptop_data["course_difficulty"], prefix="difficulty")
type_dummies = pd.get_dummies(laptop_data["course_type"], prefix="type")

# Combine all covariates
all_covariates = pd.concat([covariates_df, course_dummies, type_dummies], axis=1)

covariates = CovariateData(values=all_covariates, names=list(all_covariates.columns))

print(f"Controlling for {len(all_covariates.columns)} covariates:")
print("• Baseline ability, tech comfort, engagement")
print("• Course difficulty and type")

# Check covariate balance before adjustment
print("\n⚖️ COVARIATE BALANCE CHECK:")
balance_result = check_covariate_balance(treatment, covariates)
balance_table = balance_result["balance_table"]

print("Variables with large imbalances:")
imbalanced = balance_table[abs(balance_table["SMD"]) > 0.2]
for idx, row in imbalanced.iterrows():
    print(f"• {row['Variable']}: SMD = {row['SMD']:.3f}")

# Estimate causal effect using multiple methods
print("\n🔬 CAUSAL ESTIMATION METHODS:")

# Method 1: G-computation
g_comp = GComputationEstimator(model_type="random_forest", bootstrap_samples=100)
g_comp.fit(treatment, outcome, covariates)
g_comp_effect = g_comp.estimate_ate()

print("\n1️⃣ G-COMPUTATION (Random Forest):")
print(f"📊 Causal effect: {g_comp_effect.ate:.2f} points")
print(
    f"📈 95% CI: [{g_comp_effect.confidence_interval[0]:.2f}, {g_comp_effect.confidence_interval[1]:.2f}]"
)
print(f"🎯 Significant: {g_comp_effect.is_significant}")

# Method 2: AIPW (Doubly Robust)
aipw = AIPWEstimator(
    outcome_model="random_forest", propensity_model="logistic", bootstrap_samples=100
)
aipw.fit(treatment, outcome, covariates)
aipw_effect = aipw.estimate_ate()

print("\n2️⃣ AIPW (Doubly Robust):")
print(f"📊 Causal effect: {aipw_effect.ate:.2f} points")
print(
    f"📈 95% CI: [{aipw_effect.confidence_interval[0]:.2f}, {aipw_effect.confidence_interval[1]:.2f}]"
)
print(f"🎯 Significant: {aipw_effect.is_significant}")

# Compare with naive estimate
print("\n📊 COMPARISON OF ESTIMATES:")
print(f"❌ Naive comparison: {observed_difference:.2f} points")
print(f"🔬 G-computation: {g_comp_effect.ate:.2f} points")
print(f"🛡️ AIPW (doubly robust): {aipw_effect.ate:.2f} points")

# Calculate bias correction
bias_corrected = observed_difference - aipw_effect.ate
print("\n🎯 BIAS ANALYSIS:")
print(f"📉 Selection bias: {bias_corrected:.2f} points")
print(
    f"📊 Bias correction: {abs(bias_corrected/observed_difference)*100:.1f}% of observed effect"
)

if abs(aipw_effect.ate) < abs(observed_difference):
    print("\n✨ REVELATION: The causal effect is smaller than the raw correlation!")
    if aipw_effect.ate > 0 and observed_difference < 0:
        print(
            "🔄 Direction reversal! Laptops might actually HELP when properly analyzed!"
        )
    elif abs(aipw_effect.ate) < 2:  # Small effect
        print("📊 The true effect is much smaller than it appeared!")

# Analyze heterogeneous effects
print("\n🎭 HETEROGENEOUS EFFECTS ANALYSIS:")

# Effect by engagement level
high_engagement = laptop_data[laptop_data["engagement_level"] > 0.7]
low_engagement = laptop_data[laptop_data["engagement_level"] <= 0.5]

high_eng_effect = (
    high_engagement[high_engagement["uses_laptop"] == 1]["test_score"].mean()
    - high_engagement[high_engagement["uses_laptop"] == 0]["test_score"].mean()
)
low_eng_effect = (
    low_engagement[low_engagement["uses_laptop"] == 1]["test_score"].mean()
    - low_engagement[low_engagement["uses_laptop"] == 0]["test_score"].mean()
)

print(f"🔥 High engagement students: {high_eng_effect:.1f} points")
print(f"😴 Low engagement students: {low_eng_effect:.1f} points")

if high_eng_effect > low_eng_effect:
    print("\n💡 KEY INSIGHT: Laptops help engaged students but hurt distracted ones!")
    print("🎯 This explains the heterogeneous effects!")

print("\n🏆 CAUSAL DETECTIVE CONCLUSION:")
print("🕵️ The raw comparison was misleading due to selection bias")
print("🎯 True causal effects are more nuanced and context-dependent")
print("🧠 This is why we need to 'see different' - control for confounders!")

## Case Study 3: The Diversity Paradox in Corporate Performance

**The Apparent Pattern**: Companies with diverse leadership teams are more profitable.

**The Quick Conclusion**: Diversity causes higher profits!

**The Innovation**: Let's uncover the complex causal mechanisms and confounders at play.

In [ ]:
# Case Study 3: Corporate Diversity and Performance - Complex Causality


def generate_corporate_diversity_data(n_companies=300):
    """
    Generate corporate data with complex causal relationships
    between diversity and performance
    """
    # Company characteristics (confounders)
    company_size = np.random.lognormal(8, 1.5, n_companies)  # Log-normal distribution
    company_age = np.random.gamma(2, 10, n_companies)  # Gamma distribution for age
    industry_innovation = np.random.beta(
        2, 3, n_companies
    )  # Innovation level by industry
    market_competitiveness = np.random.uniform(0.2, 0.9, n_companies)

    # Geographic and cultural factors
    urban_location = np.random.binomial(1, 0.6, n_companies)  # Urban vs rural
    coastal_location = np.random.binomial(1, 0.4, n_companies)  # Coastal premium

    # CEO and leadership characteristics
    ceo_experience = np.random.gamma(2, 5, n_companies)  # Years of experience
    leadership_quality = np.random.beta(3, 2, n_companies)  # Leadership quality score

    # Diversity is NOT randomly assigned!
    # It's influenced by:
    # 1. Location (urban/coastal areas more diverse)
    # 2. Industry innovation (innovative industries embrace diversity)
    # 3. Company size (larger companies have more diversity)
    # 4. Leadership quality (better leaders value diversity)

    diversity_propensity = (
        0.2  # Base level
        + 0.3 * urban_location  # Urban areas more diverse
        + 0.2 * coastal_location  # Coastal areas more diverse
        + 0.3 * industry_innovation  # Innovative industries
        + 0.1 * np.log(company_size) / 10  # Larger companies
        + 0.2 * leadership_quality  # Quality leaders value diversity
        + np.random.normal(0, 0.1, n_companies)
    )

    diversity_score = np.clip(diversity_propensity, 0.1, 0.9)  # 0-1 scale

    # True causal mechanisms for diversity effect:
    # 1. Direct innovation benefit: +5% profit
    # 2. Market expansion benefit: +3% profit
    # 3. BUT communication costs: -2% profit
    # 4. Interaction with innovation: diversity helps more in innovative industries

    diversity_innovation_effect = 5 * diversity_score  # Innovation benefit
    diversity_market_effect = 3 * diversity_score  # Market expansion
    diversity_communication_cost = -2 * diversity_score  # Communication overhead
    diversity_innovation_interaction = (
        4 * diversity_score * industry_innovation
    )  # Synergy

    # Profit margin (our outcome of interest)
    base_profit = 8  # Base profit margin %

    profit_margin = (
        base_profit
        + diversity_innovation_effect  # Diversity innovation benefit
        + diversity_market_effect  # Market expansion benefit
        + diversity_communication_cost  # Communication costs
        + diversity_innovation_interaction  # Synergy effect
        + 3 * leadership_quality  # Leadership quality matters
        + 2 * industry_innovation  # Industry innovation helps
        + 1 * np.log(company_size) / 10  # Size advantages
        + 2 * market_competitiveness  # Competition drives efficiency
        + 1 * urban_location  # Urban premium
        + np.random.normal(0, 2, n_companies)
    )  # Random variation

    profit_margin = np.clip(profit_margin, 1, 25)  # Realistic range

    # Create categorical variables for analysis
    high_diversity = (diversity_score > np.median(diversity_score)).astype(int)
    large_company = (company_size > np.median(company_size)).astype(int)
    innovative_industry = (industry_innovation > 0.6).astype(int)

    return pd.DataFrame(
        {
            "diversity_score": diversity_score,
            "high_diversity": high_diversity,
            "profit_margin": profit_margin,
            "company_size": company_size,
            "large_company": large_company,
            "company_age": company_age,
            "industry_innovation": industry_innovation,
            "innovative_industry": innovative_industry,
            "market_competitiveness": market_competitiveness,
            "urban_location": urban_location,
            "coastal_location": coastal_location,
            "ceo_experience": ceo_experience,
            "leadership_quality": leadership_quality,
            "diversity_propensity": diversity_propensity,
        }
    ), {
        "innovation_effect": 5,
        "market_effect": 3,
        "communication_cost": -2,
        "net_effect": 6,  # 5 + 3 - 2
    }


# Generate the data
corp_data, true_effects = generate_corporate_diversity_data(300)

print("🏢 Corporate Diversity and Performance Dataset Generated")
print(f"True net diversity effect: +{true_effects['net_effect']}% profit margin")
print("But diversity is not randomly assigned!")
print("\nData preview:")
print(corp_data.head())

# Calculate observed correlation
diversity_correlation = np.corrcoef(
    corp_data["diversity_score"], corp_data["profit_margin"]
)[0, 1]
high_div_companies = corp_data[corp_data["high_diversity"] == 1]
low_div_companies = corp_data[corp_data["high_diversity"] == 0]
observed_difference = (
    high_div_companies["profit_margin"].mean()
    - low_div_companies["profit_margin"].mean()
)

print("\n📊 OBSERVED PATTERNS:")
print(f"📈 Diversity-Profit correlation: r = {diversity_correlation:.3f}")
print(
    f"🏆 High diversity companies: {high_div_companies['profit_margin'].mean():.1f}% profit"
)
print(
    f"📉 Low diversity companies: {low_div_companies['profit_margin'].mean():.1f}% profit"
)
print(f"⬆️ Raw difference: {observed_difference:.1f} percentage points")
print("✨ Diversity appears to boost profits!")

# Visualize the complex relationships
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Main relationship
axes[0, 0].scatter(corp_data["diversity_score"], corp_data["profit_margin"], alpha=0.6)
z = np.polyfit(corp_data["diversity_score"], corp_data["profit_margin"], 1)
p = np.poly1d(z)
axes[0, 0].plot(
    corp_data["diversity_score"], p(corp_data["diversity_score"]), "r--", alpha=0.8
)
axes[0, 0].set_title(f"Diversity vs Profit Margin\nr = {diversity_correlation:.3f}")
axes[0, 0].set_xlabel("Diversity Score")
axes[0, 0].set_ylabel("Profit Margin (%)")

# Confounder 1: Company size
size_diversity_corr = np.corrcoef(
    corp_data["company_size"], corp_data["diversity_score"]
)[0, 1]
size_profit_corr = np.corrcoef(corp_data["company_size"], corp_data["profit_margin"])[
    0, 1
]
axes[0, 1].scatter(
    np.log(corp_data["company_size"]), corp_data["diversity_score"], alpha=0.6
)
axes[0, 1].set_title(f"Size → Diversity\nr = {size_diversity_corr:.3f}")
axes[0, 1].set_xlabel("Log(Company Size)")
axes[0, 1].set_ylabel("Diversity Score")

# Confounder 2: Innovation level
innovation_diversity_corr = np.corrcoef(
    corp_data["industry_innovation"], corp_data["diversity_score"]
)[0, 1]
innovation_profit_corr = np.corrcoef(
    corp_data["industry_innovation"], corp_data["profit_margin"]
)[0, 1]
axes[0, 2].scatter(
    corp_data["industry_innovation"], corp_data["diversity_score"], alpha=0.6
)
axes[0, 2].set_title(f"Innovation → Diversity\nr = {innovation_diversity_corr:.3f}")
axes[0, 2].set_xlabel("Industry Innovation")
axes[0, 2].set_ylabel("Diversity Score")

# Geographic patterns
urban_div = corp_data.groupby("urban_location")["diversity_score"].mean()
coastal_div = corp_data.groupby("coastal_location")["diversity_score"].mean()
axes[1, 0].bar(["Rural", "Urban"], urban_div, alpha=0.8)
axes[1, 0].set_title("Diversity by Location")
axes[1, 0].set_ylabel("Average Diversity Score")

# Leadership quality patterns
leadership_diversity_corr = np.corrcoef(
    corp_data["leadership_quality"], corp_data["diversity_score"]
)[0, 1]
axes[1, 1].scatter(
    corp_data["leadership_quality"], corp_data["diversity_score"], alpha=0.6
)
axes[1, 1].set_title(f"Leadership → Diversity\nr = {leadership_diversity_corr:.3f}")
axes[1, 1].set_xlabel("Leadership Quality")
axes[1, 1].set_ylabel("Diversity Score")

# Interaction effects
innovative_companies = corp_data[corp_data["innovative_industry"] == 1]
traditional_companies = corp_data[corp_data["innovative_industry"] == 0]
innov_corr = np.corrcoef(
    innovative_companies["diversity_score"], innovative_companies["profit_margin"]
)[0, 1]
trad_corr = np.corrcoef(
    traditional_companies["diversity_score"], traditional_companies["profit_margin"]
)[0, 1]

axes[1, 2].scatter(
    innovative_companies["diversity_score"],
    innovative_companies["profit_margin"],
    alpha=0.6,
    label=f"Innovative (r={innov_corr:.2f})",
)
axes[1, 2].scatter(
    traditional_companies["diversity_score"],
    traditional_companies["profit_margin"],
    alpha=0.6,
    label=f"Traditional (r={trad_corr:.2f})",
)
axes[1, 2].set_title("Diversity Effect by Industry Type")
axes[1, 2].set_xlabel("Diversity Score")
axes[1, 2].set_ylabel("Profit Margin (%)")
axes[1, 2].legend()

plt.tight_layout()
plt.show()

print("\n🔍 CONFOUNDING ANALYSIS:")
print(f"🏗️ Company size ↔ Diversity: r = {size_diversity_corr:.3f}")
print(f"💡 Innovation ↔ Diversity: r = {innovation_diversity_corr:.3f}")
print(f"👔 Leadership ↔ Diversity: r = {leadership_diversity_corr:.3f}")
print(f"🌆 Urban companies have {urban_div[1]:.2f} vs {urban_div[0]:.2f} diversity")
print("\n🎯 Key insight: Diversity is correlated with many success factors!")

In [ ]:
# Advanced Causal Analysis: Unraveling Complex Causality

print("🕵️ ADVANCED CAUSAL INVESTIGATION: Corporate Diversity Effects")

# Prepare comprehensive covariate set
covariates_corp = corp_data[
    [
        "company_size",
        "company_age",
        "industry_innovation",
        "market_competitiveness",
        "urban_location",
        "coastal_location",
        "ceo_experience",
        "leadership_quality",
    ]
].copy()

# Log-transform company size for better modeling
covariates_corp["log_company_size"] = np.log(covariates_corp["company_size"])
covariates_corp = covariates_corp.drop("company_size", axis=1)

treatment_corp = TreatmentData(
    values=corp_data["high_diversity"], name="high_diversity", treatment_type="binary"
)

outcome_corp = OutcomeData(
    values=corp_data["profit_margin"], name="profit_margin", outcome_type="continuous"
)

covariates_corp_data = CovariateData(
    values=covariates_corp, names=list(covariates_corp.columns)
)

print(f"Controlling for {len(covariates_corp.columns)} confounders:")
for col in covariates_corp.columns:
    print(f"• {col}")

# Check balance
print("\n⚖️ BALANCE CHECK BEFORE ADJUSTMENT:")
balance_result_corp = check_covariate_balance(treatment_corp, covariates_corp_data)
balance_table_corp = balance_result_corp["balance_table"]

severe_imbalances = balance_table_corp[abs(balance_table_corp["SMD"]) > 0.3]
print("Variables with severe imbalances (SMD > 0.3):")
for idx, row in severe_imbalances.iterrows():
    print(f"• {row['Variable']}: SMD = {row['SMD']:.3f}")

# Multiple causal estimation methods
print("\n🔬 CAUSAL ESTIMATION WITH MULTIPLE METHODS:")

# Method 1: Linear G-computation
g_comp_linear = GComputationEstimator(model_type="linear", bootstrap_samples=100)
g_comp_linear.fit(treatment_corp, outcome_corp, covariates_corp_data)
linear_effect = g_comp_linear.estimate_ate()

# Method 2: Random Forest G-computation
g_comp_rf = GComputationEstimator(model_type="random_forest", bootstrap_samples=100)
g_comp_rf.fit(treatment_corp, outcome_corp, covariates_corp_data)
rf_effect = g_comp_rf.estimate_ate()

# Method 3: AIPW
aipw_corp = AIPWEstimator(
    outcome_model="random_forest", propensity_model="logistic", bootstrap_samples=100
)
aipw_corp.fit(treatment_corp, outcome_corp, covariates_corp_data)
aipw_corp_effect = aipw_corp.estimate_ate()

# Method 4: Causal Forest (heterogeneous effects)
try:
    causal_forest = CausalForestEstimator(
        n_estimators=100,
        bootstrap_samples=50,  # Reduced for speed
    )
    causal_forest.fit(treatment_corp, outcome_corp, covariates_corp_data)
    forest_effect = causal_forest.estimate_ate()
    has_forest = True
except Exception as e:
    print(f"⚠️ Causal Forest not available: {e}")
    has_forest = False

# Present results
print("\n📊 CAUSAL EFFECT ESTIMATES:")
print(f"❌ Naive comparison: +{observed_difference:.2f} percentage points")
print(
    f"📐 Linear G-computation: +{linear_effect.ate:.2f} pp [CI: {linear_effect.confidence_interval[0]:.2f}, {linear_effect.confidence_interval[1]:.2f}]"
)
print(
    f"🌳 Random Forest G-comp: +{rf_effect.ate:.2f} pp [CI: {rf_effect.confidence_interval[0]:.2f}, {rf_effect.confidence_interval[1]:.2f}]"
)
print(
    f"🛡️ AIPW (doubly robust): +{aipw_corp_effect.ate:.2f} pp [CI: {aipw_corp_effect.confidence_interval[0]:.2f}, {aipw_corp_effect.confidence_interval[1]:.2f}]"
)
if has_forest:
    print(
        f"🌲 Causal Forest: +{forest_effect.ate:.2f} pp [CI: {forest_effect.confidence_interval[0]:.2f}, {forest_effect.confidence_interval[1]:.2f}]"
    )

print(f"🎯 True net effect: +{true_effects['net_effect']:.0f} percentage points")

# Analyze bias
best_estimate = aipw_corp_effect.ate  # Use doubly robust estimate
selection_bias = observed_difference - best_estimate
bias_percentage = (selection_bias / observed_difference) * 100

print("\n🎯 BIAS DECOMPOSITION:")
print(
    f"📊 Selection bias: +{selection_bias:.2f} pp ({bias_percentage:.1f}% of observed effect)"
)
print(f"🔍 True causal effect: +{best_estimate:.2f} pp")

if selection_bias > 1:
    print("\n⚠️ MAJOR FINDING: Most of the observed effect is due to selection bias!")
    print("🏢 Successful companies are more likely to be diverse AND profitable")
    print("✨ But diversity still has a genuine causal effect!")

# Heterogeneous effects analysis
print("\n🎭 HETEROGENEOUS EFFECTS:")

# Effect by innovation level
innovative = corp_data[corp_data["innovative_industry"] == 1]
traditional = corp_data[corp_data["innovative_industry"] == 0]

innov_effect = (
    innovative[innovative["high_diversity"] == 1]["profit_margin"].mean()
    - innovative[innovative["high_diversity"] == 0]["profit_margin"].mean()
)
trad_effect = (
    traditional[traditional["high_diversity"] == 1]["profit_margin"].mean()
    - traditional[traditional["high_diversity"] == 0]["profit_margin"].mean()
)

print(f"💡 Innovative industries: +{innov_effect:.2f} pp")
print(f"🏭 Traditional industries: +{trad_effect:.2f} pp")

if innov_effect > trad_effect + 1:
    print("\n🔥 KEY INSIGHT: Diversity has stronger effects in innovative industries!")
    print("🎯 This matches our theoretical expectation!")

print("\n🏆 CAUSAL DETECTIVE CONCLUSIONS:")
print("1️⃣ Raw correlation overstates the diversity effect due to selection bias")
print("2️⃣ Successful companies are more likely to embrace diversity")
print("3️⃣ Diversity has genuine causal benefits, but they're smaller than they appear")
print("4️⃣ Effects are heterogeneous - stronger in innovative contexts")
print("5️⃣ Multiple confounders create complex causal patterns")
print("\n💡 This is why 'seeing different' means controlling for confounders!")

## The Innovator's Diagnostic Toolkit

### 🔍 The Causal Detective's Checklist

When you encounter an apparent causal relationship, ask yourself:

#### 1. Selection Bias Detection
- **Who or what gets the treatment?**
- **Is treatment assignment random or systematic?**
- **What characteristics predict treatment assignment?**

#### 2. Confounder Hunting
- **What other variables affect both treatment and outcome?**
- **Are there seasonal, geographic, or temporal patterns?**
- **What institutional or structural factors create correlations?**

#### 3. Mechanism Investigation
- **How exactly would the treatment cause the outcome?**
- **Are there intermediate steps in the causal chain?**
- **Could the relationship work in reverse?**

#### 4. Heterogeneity Analysis
- **Does the effect vary across subgroups?**
- **When does the treatment work vs. not work?**
- **What moderates the relationship?**

### 🛠️ Diagnostic Tools You've Mastered

1. **Partial Correlation Analysis**: Remove the influence of confounders
2. **Covariate Balance Checks**: Detect selection bias patterns
3. **Multiple Estimation Methods**: Triangulate causal effects
4. **Heterogeneous Effects**: Understand when effects vary
5. **Sensitivity Analysis**: Test robustness to assumptions

In [ ]:
# The Innovator's Diagnostic Function


def causal_diagnostic_suite(
    treatment_var, outcome_var, data, potential_confounders=None
):
    """
    Comprehensive diagnostic suite for causal relationships
    """
    print("🔍 CAUSAL DIAGNOSTIC SUITE ACTIVATED")
    print("=" * 50)

    results = {}

    # 1. Raw correlation
    raw_corr = np.corrcoef(data[treatment_var], data[outcome_var])[0, 1]
    results["raw_correlation"] = raw_corr

    print("\n1️⃣ RAW RELATIONSHIP:")
    print(f"📊 Correlation: r = {raw_corr:.4f}")

    if abs(raw_corr) > 0.5:
        print("🚨 Strong correlation detected - investigate further!")
    elif abs(raw_corr) > 0.3:
        print("⚠️ Moderate correlation - could be genuine or spurious")
    else:
        print("📉 Weak correlation - may not be meaningful")

    # 2. Distribution analysis
    print("\n2️⃣ DISTRIBUTION ANALYSIS:")
    if data[treatment_var].nunique() == 2:  # Binary treatment
        treated = data[data[treatment_var] == 1][outcome_var]
        control = data[data[treatment_var] == 0][outcome_var]
        diff = treated.mean() - control.mean()
        print(f"📈 Treatment group mean: {treated.mean():.2f}")
        print(f"📉 Control group mean: {control.mean():.2f}")
        print(f"⚡ Raw difference: {diff:.2f}")
        results["raw_difference"] = diff

    # 3. Confounder analysis
    if potential_confounders:
        print("\n3️⃣ CONFOUNDER ANALYSIS:")
        confounding_strength = []

        for confounder in potential_confounders:
            if confounder in data.columns:
                # Correlation with treatment
                conf_treat_corr = np.corrcoef(data[confounder], data[treatment_var])[
                    0, 1
                ]
                # Correlation with outcome
                conf_outcome_corr = np.corrcoef(data[confounder], data[outcome_var])[
                    0, 1
                ]
                # Confounding strength = product of correlations
                conf_strength = abs(conf_treat_corr * conf_outcome_corr)
                confounding_strength.append(
                    (confounder, conf_strength, conf_treat_corr, conf_outcome_corr)
                )

        # Sort by confounding strength
        confounding_strength.sort(key=lambda x: x[1], reverse=True)

        print("🎯 Potential confounders (ranked by strength):")
        for conf, strength, treat_corr, out_corr in confounding_strength[:5]:
            print(
                f"• {conf}: Strength = {strength:.3f} (r_treat={treat_corr:.3f}, r_outcome={out_corr:.3f})"
            )

        results["top_confounders"] = confounding_strength[:3]

    # 4. Partial correlation (if confounders provided)
    if potential_confounders and len(potential_confounders) > 0:
        print("\n4️⃣ PARTIAL CORRELATION ANALYSIS:")
        try:
            from sklearn.linear_model import LinearRegression

            # Select available confounders
            available_confounders = [
                c for c in potential_confounders if c in data.columns
            ]
            if available_confounders:
                X_conf = data[available_confounders].fillna(
                    data[available_confounders].mean()
                )

                # Regress treatment on confounders
                reg_treat = LinearRegression().fit(X_conf, data[treatment_var])
                treat_residuals = data[treatment_var] - reg_treat.predict(X_conf)

                # Regress outcome on confounders
                reg_outcome = LinearRegression().fit(X_conf, data[outcome_var])
                outcome_residuals = data[outcome_var] - reg_outcome.predict(X_conf)

                # Partial correlation
                partial_corr = np.corrcoef(treat_residuals, outcome_residuals)[0, 1]
                results["partial_correlation"] = partial_corr

                print(f"📊 Raw correlation: {raw_corr:.4f}")
                print(f"🎯 Partial correlation: {partial_corr:.4f}")

                reduction = abs(raw_corr) - abs(partial_corr)
                pct_reduction = (
                    (reduction / abs(raw_corr)) * 100 if raw_corr != 0 else 0
                )

                print(
                    f"⬇️ Confounding reduction: {reduction:.4f} ({pct_reduction:.1f}%)"
                )

                if pct_reduction > 50:
                    print(
                        "🚨 MAJOR CONFOUNDING DETECTED! Over 50% of correlation explained by confounders"
                    )
                elif pct_reduction > 25:
                    print("⚠️ Moderate confounding detected")
                else:
                    print("✅ Relationship appears robust to these confounders")

        except Exception as e:
            print(f"⚠️ Partial correlation analysis failed: {e}")

    # 5. Diagnostic conclusions
    print("\n5️⃣ DIAGNOSTIC CONCLUSIONS:")

    if "partial_correlation" in results:
        if abs(results["partial_correlation"]) < abs(results["raw_correlation"]) * 0.5:
            print(
                "🔍 LIKELY SPURIOUS: Relationship weakens substantially when controlling for confounders"
            )
        elif (
            abs(results["partial_correlation"]) > abs(results["raw_correlation"]) * 0.8
        ):
            print("✨ LIKELY GENUINE: Relationship robust to confounders")
        else:
            print(
                "🤔 MIXED EVIDENCE: Some confounding present but relationship persists"
            )

    print("\n💡 RECOMMENDATIONS:")
    if abs(raw_corr) > 0.3:
        print("• Collect more potential confounders")
        print("• Use causal inference methods (IV, RDD, DiD)")
        print("• Look for natural experiments")
        print("• Conduct robustness checks")
    else:
        print("• Consider if relationship is meaningful")
        print("• Look for non-linear effects")
        print("• Check for interaction effects")

    return results


# Example usage with our ice cream data
print("🍦 EXAMPLE: Diagnosing Ice Cream and Drowning Relationship")
ice_cream_results = causal_diagnostic_suite(
    treatment_var="ice_cream_sales",
    outcome_var="drowning_deaths",
    data=ice_cream_data,
    potential_confounders=["temperature", "marketing_spend", "swimming_activity"],
)

print("\n" + "=" * 50)
print("🎯 This diagnostic suite helps you 'see different' by:")
print("• Systematically checking for confounders")
print("• Quantifying spurious correlation")
print("• Providing actionable next steps")
print("• Building your causal intuition")

## Challenge Exercise: Become a Causal Detective

Now it's your turn to practice "seeing different". Choose one of these scenarios and apply the diagnostic toolkit:

### Scenario Options

1. **The Coffee Productivity Paradox**
   - *Observed*: Employees who drink more coffee are more productive
   - *Question*: Does coffee cause productivity or vice versa?

2. **The Social Media Depression Link**
   - *Observed*: Heavy social media users report higher depression rates
   - *Question*: Does social media cause depression?

3. **The Expensive Wine Quality Correlation**
   - *Observed*: More expensive wines get higher ratings
   - *Question*: Does price influence perception of quality?

4. **The Exercise Happiness Connection**
   - *Observed*: People who exercise regularly are happier
   - *Question*: Does exercise cause happiness?

### Your Detective Mission

For your chosen scenario:
1. **List potential confounders**
2. **Identify selection mechanisms**
3. **Propose diagnostic tests**
4. **Design an ideal study**
5. **Predict what you'd find**

In [ ]:
# Your Detective Challenge Template

detective_template = """
🕵️ CAUSAL DETECTIVE CHALLENGE
================================

📋 CHOSEN SCENARIO: [Fill in your choice]

🎯 RESEARCH QUESTION:
[What is the causal question you're investigating?]

🔍 POTENTIAL CONFOUNDERS:
1. [List variables that might affect both treatment and outcome]
2. 
3. 
4. 
5. 

⚖️ SELECTION MECHANISMS:
[How do people/units get assigned to treatment? Is it random or systematic?]

🧪 DIAGNOSTIC TESTS I WOULD RUN:
1. [What analyses would help you detect confounding?]
2. 
3. 

🎛️ IDEAL STUDY DESIGN:
[If you could design the perfect study, what would it look like?]

🔮 PREDICTIONS:
• Naive estimate: [What would raw correlation show?]
• After controlling for confounders: [What would causal effect be?]
• Heterogeneous effects: [Would effects vary across groups?]

💡 KEY INSIGHTS:
[What would this teach us about 'seeing different'?]
"""

print(detective_template)

# Example: Coffee Productivity Analysis
print("\n📝 EXAMPLE ANALYSIS: Coffee Productivity Paradox")
print("=" * 50)

coffee_analysis = """
🎯 RESEARCH QUESTION:
Does coffee consumption causally increase workplace productivity?

🔍 POTENTIAL CONFOUNDERS:
1. Work schedule (early birds drink more coffee AND are more productive)
2. Job type (demanding jobs → more coffee AND higher productivity pressure)
3. Personality (ambitious people drink more coffee AND work harder)
4. Sleep quality (poor sleep → more coffee, but worse productivity)
5. Workplace culture (competitive environments encourage both)

⚖️ SELECTION MECHANISMS:
Coffee consumption isn't random! It's driven by:
• Personal preferences and genetics
• Work demands and stress levels  
• Social norms and peer influence
• Access and cost considerations

🧪 DIAGNOSTIC TESTS:
1. Compare coffee drinkers vs non-drinkers on work schedules
2. Look at productivity during coffee shortages/strikes
3. Analyze within-person variation (same person, different coffee days)
4. Check if relationship holds across different job types

🎛️ IDEAL STUDY DESIGN:
Randomized trial: Randomly assign workers to receive free coffee vs decaf
for 3 months, measure productivity blind to treatment assignment.
BUT: Ethical issues with caffeine withdrawal!

ALTERNATIVE: Instrumental variable - use coffee shop closures/openings
near workplaces as random variation in coffee access.

🔮 PREDICTIONS:
• Naive estimate: +15% productivity for coffee drinkers
• After controlling for confounders: +5% productivity
• Heterogeneous effects: Stronger for morning workers, weaker for anxious people

💡 KEY INSIGHTS:
• Most "productivity" correlation is selection bias
• True causal effect exists but is smaller
• Context matters - coffee helps some people more than others
• Always ask "Who chooses the treatment and why?"
"""

print(coffee_analysis)

print("\n🏆 YOUR TURN!")
print("Choose a scenario and fill out the template above.")
print("Practice 'seeing different' by questioning apparent relationships!")

## Summary: The Art of Seeing Different

### What You've Mastered

1. **Spurious Correlation Detection**
   - Identified hidden confounders (temperature in ice cream example)
   - Used partial correlation to unmask true relationships
   - Learned to question "obvious" associations

2. **Selection Bias Recognition**
   - Discovered how non-random treatment assignment creates bias
   - Analyzed complex selection mechanisms in corporate diversity
   - Controlled for confounders using advanced methods

3. **Mechanism Thinking**
   - Explored heterogeneous effects across contexts
   - Identified when treatments work vs. don't work
   - Understood complex causal pathways

4. **Diagnostic Skills**
   - Built a comprehensive diagnostic toolkit
   - Learned to systematically test causal claims
   - Developed intuition for confounding patterns

### The Innovator's Mindset

**Traditional thinking**: "These variables are correlated, so one causes the other."

**Revolutionary thinking**: "What hidden factors create this correlation? How can I isolate the true causal relationship?"

### Key Principles for Seeing Different

1. **Question Everything**: Every correlation has a story behind it
2. **Hunt for Confounders**: Look for variables that predict both treatment and outcome
3. **Think About Selection**: Ask who gets treated and why
4. **Test Robustness**: Use multiple methods to verify findings
5. **Expect Heterogeneity**: Effects vary across contexts and populations

### Next Steps in Your Innovation Journey

Continue developing your causal vision with:
- **Tutorial 3**: The Crazy Idea - Using ML for causal inference
- **Tutorial 4**: Change Things - From analysis to intervention
- **Tutorial 5**: Push Forward - Advanced causal methods

---

*"While others see complexity, innovators see clarity." You now have the vision to see what others miss. Keep questioning, keep investigating, and keep seeing different!*